In [1]:
# import DPTool as dp 
# need to comment out until update package in order to get right config file 
from pipeline import DataProcessing as dp_dev

c:\Users\krodriguez0109\AppData\Local\anaconda3\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\krodriguez0109\AppData\Local\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\krodriguez0109\AppData\Local\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
dp_class = dp.DataProcessing('./data/sasol_data_sample.csv','OXO-5FI696 Augusta','OXO-5FI696 Augusta','max') 
final_df = dp_class.pipeline()

NameError: name 'dp' is not defined

In [2]:
dp_git =  dp_dev('./data/sasol_data_sample.csv','OXO-5FI696 Augusta')
cleandf = dp_git.pipeline()

Loading Data...
Getting Size...
Data size: (100, 59)
Summarizing Type...
Data types summary:
 float64    59
Name: count, dtype: int64
Summarizing Missing Data...
Missing data summary:
OXO-5FIC606A Augusta    100
OXO-5FIC611B Augusta    100
OXO-5LI654F Augusta     100
dtype: int64


Removing Duplicates...
Handling Missing Data...
# rows dropped with missing values in the target variable: 0
Dropped columns: ['OXO-5FIC606A Augusta', 'OXO-5FIC611B Augusta', 'OXO-5LI654F Augusta']
Filled missing values using mean
Missing data summary:
Series([], dtype: int64)
Generating Anomaly Report...
./Data_Cleansing/OXO-5FI696 Augusta_anomaly_report already exists, trying again...
[WinError 183] Cannot create a file when that file already exists: './Data_Cleansing/OXO-5FI696 Augusta_anomaly_report\\xlsx'
Selected Features are OXO-5FI696 Augusta, OXO-5FI635C Augusta, OXO-5FIC602 Augusta, OXO-5FIC612B Augusta, OXO-5FIC622C Augusta, OXO-5FIC636C Augusta, OXO-5TIC602 Augusta, OXO-5TIC603 Augusta
Finding bounds...
Found Bounds!
Index(['OXO-5FI696 Augusta'], dtype='object')
OXO-5FI696 Augusta
Done
Detecting Outliers...
2 outliers detected in OXO-5FI696 Augusta
3 outliers detected in OXO-5FIC612A Augusta
3 outliers detected in OXO-5FIC618A Augusta
3 outliers detected in OXO-5FIC618D Aug

Selecting Features...


  0%|          | 0/20 [00:00<?, ?it/s]

9 attributes confirmed important: ['OXO-5FQI691B Augusta', 'OXO-5LI633B Augusta', 'OXO-5LI651F Augusta', 'OXO-5LI654E Augusta', 'OXO-5FIC636C Augusta', 'OXO-5FIC602 Augusta', 'OXO-5TIC603 Augusta', 'OXO-5FIC612B Augusta', 'OXO-5FIC618B Augusta']
37 attributes confirmed unimportant: ['OXO-_5FI658A Augusta', 'OXO-5TIC602 Augusta', 'OXO-5LI650F Augusta', 'OXO-5FIC601 Augusta', 'OXO-5FIC622A Augusta', 'OXO-5RIC606_Y Augusta', 'OXO-5FIC622B Augusta', 'OXO-5FIC609A Augusta', 'OXO-5FIC622C Augusta', 'OXO-5TIC605 Augusta', 'OXO-5FIC612A Augusta', 'OXO-5FIC618C Augusta', 'OXO-5PIC605A Augusta', 'OXO-5LI657F Augusta', 'OXO-5TIC601 Augusta', 'OXO-5LI658F Augusta', 'OXO-_5FI659A Augusta', 'OXO-5LI651E Augusta', 'OXO-5FIC607 Augusta', 'OXO-5LI652F Augusta', 'OXO-5FIC636B Augusta', 'OXO-5LI658E Augusta', 'OXO-5FIC618D Augusta', 'OXO-5LI653E Augusta', 'OXO-5TIC604 Augusta', 'OXO-5TIC002 Augusta', 'OXO-5TIC606 Augusta', 'OXO-5FIC609B Augusta', 'OXO-_5FI660A Augusta', 'OXO-5FIC603 Augusta', 'OXO-5TIC60